In [1]:
!pip -q install transformers==4.34.0
!pip -q install accelerate==0.23.0
!pip -q install flash-attn==2.3.3 --no-build-isolation
!pip install -q -U bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 30.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 39.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 78.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 75.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 37.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 15.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 23.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 9.4 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [11]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

device = "cuda:0" # the device to load the model onto

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# Set environment variable for memory management
torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True
torch.backends.cuda.matmul.allow_fp16_reduced_precision_reduction = True
torch.backends.cudnn.benchmark = True

model_id = "amazon/MistralLite"
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, torch_dtype=torch.float16)
tokenizer = AutoTokenizer.from_pretrained(model_id)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
prompt = """
[INST] Based on the following disorder guidelines: Q1. Generalized Anxiety Disorder, Q2. Panic Disorder, Q3. Agoraphobia, Q4. Specific Phobia, Q5. Social Anxiety Disorder, Q6. Separation Anxiety Disorder, Q7. Selective Mutism, Q8. Other Anxiety and Fear-Related Disorder, Q9. Unspecified Anxiety and Fear-Related Disorder, please write a diagnosis for the following vignette. [/INST]"""

In [8]:
import pandas as pd

path_guideline = "/content/drive/MyDrive/Capstone/Anxiety_Disorder_EF_ACG.txt"
path_vignette = "/content/drive/MyDrive/Capstone/Anxiety_Disorder_Vignettes.csv"

with open(path_guideline, 'r', encoding='utf-8') as f:
    guidelines = f.read()

vignettes = pd.read_csv(path_vignette)
vignette_1 = vignettes.loc[0, 'Description']

In [9]:
prompt_final = prompt + guidelines + vignette_1

In [14]:
messages = [
    {"role": "user", "content": prompt},
]

encodeds = tokenizer.apply_chat_template(messages, return_tensors="pt")

model_inputs = encodeds.to(device)
# model.to(device)

try:
    generated_ids = model.generate(model_inputs,
                                # max_new_tokens=1000,
                                do_sample=True,
                                temperature=0.1,
                                pad_token_id=tokenizer.eos_token_id)

    decoded = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
    print("\n".join(decoded[0].split("[/INST]")[1:]).strip())

except RuntimeError as e:
    print(e)
    print("Try again with a smaller max_new_tokens value")

Using sep_token, but it is not set yet.
Using cls_token, but it is not set yet.
Using mask_token, but it is not set yet.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1260: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1268: UserWarning: Input length of input_ids is 256, but `max_length` is set to 20. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
  warnings.warn(


In [13]:
print(decoded[0])

[INST] <<SYS>>
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
<</SYS>>


[INST] Based on the following disorder guidelines: Q1. Generalized Anxiety Disorder, Q2. Panic Disorder, Q3. Agoraphobia, Q4. Specific Phobia, Q5. Social Anxiety Disorder, Q6. Separation Anxiety Disorder, Q7. Selective Mutism, Q8. Other Anxiety and Fear-Related Disorder, Q9. Unspecified Anxiety and Fear-Related Disorder, please write a diagnosis for the following vignette. [/INST] [/INST]

A 16-year-old girl has been experiencing intense anxiety and fear for the

In [10]:
messages = [
    {"role": "user", "content": prompt},
]

encodeds = tokenizer.apply_chat_template(messages, return_tensors="pt")

model_inputs = encodeds.to(device)
# model.to(device)

try:
    generated_ids = model.generate(model_inputs,
                                max_new_tokens=1000,
                                do_sample=True,
                                temperature=0.1,
                                pad_token_id=tokenizer.eos_token_id)

    decoded = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
    print("\n".join(decoded[0].split("[/INST]")[1:]).strip())

except RuntimeError as e:
    print(e)
    print("Try again with a smaller max_new_tokens value")

Using sep_token, but it is not set yet.
Using pad_token, but it is not set yet.
Using cls_token, but it is not set yet.
Using mask_token, but it is not set yet.


Based on the vignette provided, the diagnosis for the individual would be:

Q9. Unspecified Anxiety and Fear-Related Disorder.

The individual experiences a range of anxiety and fear-related symptoms, including difficulty sleeping, irritability, and avoidance of certain situations or people. These symptoms are not specific to any one anxiety or fear-related disorder, making it difficult to diagnose a specific disorder. However, the individual's symptoms are significant enough to impair their daily functioning and quality of life.
